# Project 1b

This serves as a template which will guide you through the implementation of this task. It is advised
to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [1]:
import numpy as np
import pandas as pd

# Add any additional imports here (however, the task is solvable without using 
# any additional imports)
from sklearn import linear_model, model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, PredictionErrorDisplay

 #### Loading data

In [2]:
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())
X = data.to_numpy()

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08


#### Transform data

In [3]:
"""
Transform the 5 input features of matrix X (x_i denoting the i-th component of X) 
into 21 new features phi(X) in the following manner:
5 linear features: phi_1(X) = x_1, phi_2(X) = x_2, phi_3(X) = x_3, phi_4(X) = x_4, phi_5(X) = x_5
5 quadratic features: phi_6(X) = x_1^2, phi_7(X) = x_2^2, phi_8(X) = x_3^2, phi_9(X) = x_4^2, phi_10(X) = x_5^2
5 exponential features: phi_11(X) = exp(x_1), phi_12(X) = exp(x_2), phi_13(X) = exp(x_3), phi_14(X) = exp(x_4), phi_15(X) = exp(x_5)
5 cosine features: phi_16(X) = cos(x_1), phi_17(X) = cos(x_2), phi_18(X) = cos(x_3), phi_19(X) = cos(x_4), phi_20(X) = cos(x_5)
1 constant feature: phi_21(X)=1

Parameters
----------
X: matrix of floats, dim = (700,5), inputs with 5 features

Compute
----------
X_transformed: array of floats: dim = (700,21), transformed input with 21 features
"""
X_transformed = np.zeros((700, 21))

# TODO: Enter your code here
X_transformed = np.hstack((X, np.square(X), np.exp(X), np.cos(X), np.ones((700, 1))))
#####

assert X_transformed.shape == (700, 21)

#### Fit data (TODO)

In [4]:
"""
Use the transformed data points X_transformed and fit the linear regression on this 
transformed data. Finally, compute the weights of the fitted linear regression. 

Parameters
----------
X_transformed: array of floats: dim = (700,21), transformed input with 21 features
y: array of floats, dim = (700,), input labels)

Compute
----------
w: array of floats: dim = (21,), optimal parameters of linear regression
"""
w = np.zeros((21,))

# TODO: Enter your code here

#####

assert w.shape == (21,)

In [5]:
# square loss ; ridge, lambda = 0.1
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X_transformed, y, test_size = 0.2, random_state = 42)
reg = linear_model.Ridge(alpha=0.1, fit_intercept=False, random_state=42)
reg.fit(x_train, y_train)
pred_valid = reg.predict(x_valid)
w = reg.coef_
print("Coefficients: \n", w)
# evaluate model performance on validation data
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_valid, pred_valid)))

reg.fit(X_transformed, y)
w = reg.coef_

Coefficients: 
 [ 0.83560546 -1.0922304  -0.9373202   0.39572759 -0.06052491 -1.65909449
  1.57817549 -0.82710702  0.38750165  3.36084902 -1.18100705 -0.66577088
 -1.74241497 -0.18267602  0.90279138  0.46742226 -1.17418472  0.03076527
 -0.62545767 -2.01133209 -0.38726443]
Root Mean Squared Error: 1.8246278497465436


In [6]:
# square loss ; ridge, lambda = 0.1; stochastic average gradient descent
reg = linear_model.Ridge(alpha=0.1, fit_intercept=False, solver='saga', random_state=42)
reg.fit(x_train, y_train)
pred_valid = reg.predict(x_valid)
w = reg.coef_
print("Coefficients: \n", w)
# evaluate model performance on validation data
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_valid, pred_valid)))

reg.fit(X_transformed, y)
w = reg.coef_

Coefficients: 
 [ 0.80711335 -1.08612605 -0.93600379  0.38388339 -0.05694639 -1.67246809
  1.57525671 -0.81807855  0.38186206  3.31414926 -1.15248637 -0.67200265
 -1.74148948 -0.1713281   0.88279014  0.46378936 -1.17979813  0.01817814
 -0.62569178 -1.99705229 -0.3947176 ]
Root Mean Squared Error: 1.8245924410535572


In [7]:
# square loss ; lasso, lambda = 0.1
reg = linear_model.Lasso(alpha=0.1, fit_intercept=False, random_state=42)
reg.fit(x_train, y_train)
pred_valid = reg.predict(x_valid)
w = reg.coef_
print("Coefficients: \n", w)
# evaluate model performance on validation data
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_valid, pred_valid)))

reg.fit(X_transformed, y)
w = reg.coef_

Coefficients: 
 [ 0.          0.          0.          0.          0.         -0.
 -0.         -0.         -0.          0.         -0.07735548 -1.58858225
 -1.51381443 -0.         -0.         -1.14980985 -1.49865648 -0.02969151
 -0.         -0.58172346 -0.        ]
Root Mean Squared Error: 1.8232168615145992


In [8]:
# Huber loss ; epsilon = 4, lambda = 0.2
reg = linear_model.HuberRegressor(epsilon=4, alpha=0.2, fit_intercept=False, max_iter=1000)
reg.fit(x_train, y_train)
pred_valid = reg.predict(x_valid)
w = reg.coef_
print("Coefficients: \n", w)
# evaluate model performance on validation data
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_valid, pred_valid)))

reg.fit(X_transformed, y)
w = reg.coef_

Coefficients: 
 [ 0.4491419  -0.87807598 -0.9203283   0.29633193  0.02416857 -1.07251858
  0.83517203 -0.27380556  0.18328835  1.24471363 -0.75487491 -0.93361942
 -1.52698303 -0.17981854  0.05872438  0.06984616 -0.88500311 -0.33210305
 -0.57155137 -1.07269721 -0.46984047]
Root Mean Squared Error: 1.8253920930665348


In [9]:
# Save results in the required format
# np.savetxt("./results-huberloss-eps4-ridge0.2.csv", w, fmt="%.12f")